<a href="https://colab.research.google.com/github/jongwoonalee/jongwoonalee.github.io/blob/main/Laplacian_of_Gaussian_(LoG)_Scale_Space_Blob_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Laplacian of Gaussian (LoG) Scale-Space Blob Detector

이 코드는 이미지에서 블롭(원형 구조)을 검출하기 위한 Laplacian of Gaussian(LoG) 필터와
스케일 공간 추적 방법을 구현합니다.

컴퓨터 비전에서 '블롭'은 주변과 구분되는 원형 또는 타원형 영역을 의미합니다.
예를 들어, 해바라기 사진에서 각 해바라기의 중심부는 블롭으로 감지될 수 있습니다.

작성자: [학생 이름]
날짜: 2025년 4월 15일
"""

import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import numpy as np  # 수학 연산을 위한 유일한 라이브러리입니다. OpenCV는 사용하지 않습니다!
from matplotlib import image as mpimg
import urllib.request
from PIL import Image
import io
from IPython.display import display

def LoG(sigma=1, size=19):
    """
    지정된 시그마와 크기로 Laplacian of Gaussian(LoG) 필터를 생성합니다.

    LoG 필터는 블롭 검출에 사용되는데, 블롭의 중심에서 강한 응답을 생성합니다.
    시그마 값은 검출할 블롭의 크기를 결정하며, 일반적으로 반지름이 약 √2σ인 블롭을
    가장 잘 검출합니다.

    매개변수:
        sigma (float): 가우시안의 표준편차. 큰 sigma는 큰 블롭을 검출합니다.
        size (int): 필터의 크기(너비와 높이)

    반환값:
        numpy array: (size, size) 형태의 LoG 필터
    """
    # 필터의 크기가 홀수여야 명확한 중심점이 있습니다!!!
    # 짝수라면 홀수로 변환합니다.
    if size % 2 == 0:
        size += 1

    # 중심이 0인 좌표 그리드 생성: 예를 들어, size=5이면
    # x와 y는 [-2, -1, 0, 1, 2]가 됩니다.
    # np.meshgrid는 두 배열의 모든 조합으로 이루어진 좌표 행렬을 생성합니다.
    half_size = size // 2
    x, y = np.meshgrid(np.arange(-half_size, half_size + 1),
                       np.arange(-half_size, half_size + 1))

    # 중심으로부터의 제곱 거리 계산
    # 이는 수식에서 x^2 + y^2 부분입니다
    squared_distance = x**2 + y**2

    # LoG 필터 수식에 따라 값 계산
    # LoG(x,y) = -1/(pi*sigma^4) * (1 - (x^2+y^2)/(2*sigma^2)) * exp(-(x^2+y^2)/(2*sigma^2))
    #
    # 이 수식의 각 부분 설명:
    # 1. -1/(pi*sigma^4): 정규화 상수로, 필터의 응답 크기를 조절
    # 2. (1 - squared_distance/(2*sigma^2)): 라플라시안 연산을 나타내는 부분
    # 3. exp(-squared_distance/(2*sigma^2)): 가우시안 함수로, 중심에서 멀어질수록 값이 감소
    log_filter = -1 / (np.pi * sigma**4) * (1 - squared_distance / (2 * sigma**2)) * np.exp(-squared_distance / (2 * sigma**2))

    # 필터를 정규화하여 합이 0이 되도록 함
    # 이는 균일한 영역에서 응답이 0이 되도록 보장합니다
    # DC 성분을 제거함으로써 밝기의 절대적 수준이 아닌 변화에만 반응하게 합니다
    log_filter = log_filter - np.mean(log_filter)

    return log_filter


def display_LoG_filters():
    """
    시그마 값이 1에서 10까지 변하는 LoG 필터들을 시각화합니다.
    이를 통해 시그마 값이 필터 형태에 미치는 영향을 볼 수 있습니다.
    """
    plt.figure(figsize=(15, 10))
    for i, sigma in enumerate(range(1, 11)):
        # 현재 시그마로 LoG 필터 생성
        log_filter = LoG(sigma=sigma, size=19)

        # 필터 시각화
        plt.subplot(2, 5, i+1)
        plt.imshow(log_filter, cmap='viridis')
        plt.title(f'LoG Filter, σ={sigma}')
        plt.colorbar()

    plt.tight_layout()
    plt.show()

    # 추가 설명: 작은 시그마 값에서는 필터가 날카롭고 중심 부분이 작습니다.
    # 큰 시그마 값에서는 필터가 더 넓게 퍼지며, 중심 영역도 커집니다.
    # 이는 각각 작은 블롭과 큰 블롭을 감지하는 데 최적화되어 있음을 의미합니다.


def filterImg(img, fil):
    """
    이미지와 필터의 컨볼루션을 수행합니다.

    컨볼루션은 필터를 이미지 위에 슬라이딩하면서 각 위치에서의 곱의 합을 계산하는 과정입니다.
    이는 이미지 처리의 기본 연산으로, 에지 검출, 블러링 등 다양한 효과를 얻을 수 있습니다.

    매개변수:
        img (numpy array): 입력 이미지
        fil (numpy array): 필터 커널

    반환값:
        numpy array: 입력과 동일한 크기를 가진 필터링된 이미지
    """
    # 이미지와 필터의 크기 가져오기
    img_height, img_width = img.shape
    fil_height, fil_width = fil.shape

    # 동일한 이미지 크기를 유지하기 위해 필요한 패딩 계산
    # 필터 크기의 절반만큼 패딩을 추가하면 출력 이미지 크기가 입력과 동일해집니다
    pad_height = fil_height // 2
    pad_width = fil_width // 2

    # 이미지에 0으로 패딩 추가
    # mode='constant'는 패딩 영역을 0으로 채웁니다
    padded_img = np.pad(img, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')

    # 입력과 동일한 크기의 출력 이미지 생성
    filtered_img = np.zeros_like(img, dtype=float)

    # 컨볼루션 수행
    # 이는 외부 라이브러리 없이 구현한 간단한 방법입니다
    # 필터를 이미지 위에 슬라이딩하며 원소별 곱셈과 합을 계산합니다
    for i in range(img_height):
        for j in range(img_width):
            # 패딩된 이미지에서 관심 영역(ROI) 추출
            roi = padded_img[i:i+fil_height, j:j+fil_width]
            # 필터 적용 (원소별 곱셈 후 합계)
            filtered_img[i, j] = np.sum(roi * fil)

    return filtered_img


def trackScale(img, sigmas, size, threshold):
    """
    LoG 필터를 사용하여 여러 스케일에서 블롭을 추적합니다.

    스케일 공간 이론은 다양한 크기의 구조를 탐지하기 위해 이미지를 다양한
    스케일(시그마 값)에서 분석하는 방법입니다. 각 스케일에서 검출된 특징 중
    가장 강한 응답을 가진 것을 선택합니다.

    매개변수:
        img (numpy array): 입력 이미지
        sigmas (list): 다양한 스케일의 시그마 값 리스트
        size (int): LoG 필터의 크기
        threshold (float): 블롭 검출을 위한 임계값

    반환값:
        numpy array: 각 픽셀에서의 최대 필터 응답
    """
    # 스케일 수
    num_scales = len(sigmas)

    # 다양한 스케일에서의 필터링된 이미지를 저장할 배열 초기화
    filtered_images = np.zeros((num_scales, img.shape[0], img.shape[1]))

    # 처리 시간 측정 시작
    start_time = time.time()

    # 각 스케일에서 LoG 필터 생성 및 이미지에 적용
    for i, sigma in enumerate(sigmas):
        # LoG 필터 생성
        log_filter = LoG(sigma=sigma, size=size)
        # 이미지에 필터 적용
        filtered_images[i] = filterImg(img, log_filter)

    # 필터링 후 정규화 적용 (필터링 중이 아님)
    # 이는 극단적인 값이 결과에 영향을 미치는 것을 방지합니다

    # 모든 스케일에서 각 픽셀에 대한 최대 응답 찾기
    # 원래의 수정되지 않은 최대 응답
    max_response = np.max(filtered_images, axis=0)

    # 더 나은 임계값 적용을 위해 최대 응답 정규화 (0-1 범위로)
    max_response_norm = (max_response - np.min(max_response)) / (np.max(max_response) - np.min(max_response))

    # 임계값을 적용하여 블롭 위치 찾기
    # np.where는 조건을 만족하는 모든 위치의 인덱스를 반환합니다
    blob_locations = np.where(max_response_norm > threshold)

    # 각 픽셀에 대해 최대 응답을 제공하는 스케일(시그마 인덱스) 찾기
    scale_indices = np.argmax(filtered_images, axis=0)

    # 각 블롭에 해당하는 스케일 가져오기
    blob_scales = scale_indices[blob_locations]

    # 실제 시그마 값으로 변환
    blob_sigmas = np.array([sigmas[scale] for scale in blob_scales])

    # 입력 이미지 위에 오버레이된 블롭으로 시각화
    plt.figure(figsize=(12, 10))
    plt.imshow(img, cmap='gray')

    # 블롭을 나타내는 원 그리기
    # 각 원의 반지름은 블롭이 감지된 시그마 값에 비례합니다
    # 시각화를 위해 블롭 개수 제한 (과부하 방지)
    max_blobs_to_show = 200
    num_blobs = min(len(blob_locations[0]), max_blobs_to_show)

    # 응답 강도별로 블롭 정렬하여 가장 강한 블롭 표시
    blob_strengths = max_response[blob_locations]
    sorted_indices = np.argsort(-blob_strengths)  # 내림차순 정렬

    # 가장 강한 블롭만 표시
    for i in range(min(num_blobs, len(sorted_indices))):
        idx = sorted_indices[i]
        y, x = blob_locations[0][idx], blob_locations[1][idx]
        sigma = blob_sigmas[idx]

        # 시그마에 비례하는 반지름으로 원 그리기
        # 가시성을 위해 다른 색상 사용
        circle = plt.Circle((x, y), sigma * 1.5, color='red', fill=False, linewidth=1.5)
        plt.gca().add_patch(circle)

    plt.title(f'Detected Blobs (threshold={threshold})')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    # 처리 시간과 검출된 블롭 수 출력
    print(f"처리 시간: {time.time() - start_time:.2f}초")
    print(f"검출된 블롭 수: {len(blob_locations[0])}")

    return max_response


def load_and_preprocess_image():
    """
    해바라기 이미지를 다운로드하고 전처리합니다.
    이미지를 30% 크기로 조정하고 그레이스케일로 변환합니다.
    """
    # 해바라기 이미지 URL
    url = "https://showme.missouri.edu/wp-content/uploads/2022/09/092722Sunflowers5-940x627.jpg"

    # 이미지 다운로드
    response = urllib.request.urlopen(url)
    image_data = response.read()

    # PIL Image로 변환
    pil_img = Image.open(io.BytesIO(image_data))

    # 원본 크기의 30%로 리사이즈
    width, height = pil_img.size
    new_width, new_height = int(width * 0.3), int(height * 0.3)
    resized_img = pil_img.resize((new_width, new_height))

    # 그레이스케일로 변환
    # 'L' 모드는 8비트 픽셀의 그레이스케일 이미지를 의미합니다
    # 그레이스케일을 사용하는 이유:
    # 1. 계산 효율성 - 3개 채널 대신 1개 채널만 처리
    # 2. 블롭 검출은 주로 밝기 변화에 기반함
    # 3. 색상보다 형태에 집중하기 위함
    gray_img = resized_img.convert('L')

    # NumPy 배열로 변환
    img_array = np.array(gray_img)

    # 이미지 표시
    plt.figure(figsize=(10, 8))
    plt.imshow(img_array, cmap='gray')
    plt.title('Resized Sunflower Image (30%)')
    plt.axis('off')
    plt.show()

    return img_array


def optimize_parameters(img_array):
    """
    해바라기 이미지의 블롭 검출을 위한 최적의 매개변수를 찾습니다.
    """
    # 다양한 임계값이 블롭 검출에 미치는 영향 분석
    thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

    # 임계값 비교를 위한 플롯 설정
    plt.figure(figsize=(18, 12))

    # 정량적 비교를 위한 블롭 수 저장
    blob_counts = []

    # 각 임계값에 대해 블롭 검출하고 결과 시각화
    for i, thresh in enumerate(thresholds):
        # 매개변수 복사 후 임계값만 업데이트
        curr_sigmas = np.linspace(2, 15, 25)
        curr_filter_size = 25

        # 현재 임계값으로 검출 실행
        curr_response = trackScale(img_array, curr_sigmas, curr_filter_size, thresh)

        # 검출된 블롭 수 계산
        blob_count = len(np.where(((curr_response - np.min(curr_response)) /
                                  (np.max(curr_response) - np.min(curr_response))) > thresh)[0])
        blob_counts.append(blob_count)

        # 상단 행에 이진 맵 플롯
        plt.subplot(2, len(thresholds), i+1)
        binary_map = (curr_response > thresh).astype(np.float64)
        plt.imshow(binary_map, cmap='gray')
        plt.title(f'Threshold = {thresh}')
        plt.axis('off')

    # 임계값과 블롭 수 사이의 관계를 보여주는 플롯 추가
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, blob_counts, 'o-', linewidth=2, markersize=10)
    plt.xlabel('임계값')
    plt.ylabel('검출된 블롭 수')
    plt.title('임계값이 블롭 검출 수에 미치는 영향')
    plt.grid(True)
    plt.xticks(thresholds)
    plt.tight_layout()
    plt.show()

    # 결과 분석 및 설명
    print("임계값 분석 결과:")
    for thresh, count in zip(thresholds, blob_counts):
        print(f"임계값 {thresh}: {count}개 블롭 검출")

    print("\n분석:")
    print("임계값이 증가할수록 검출된 블롭 수가 감소합니다.")
    print("낮은 임계값(0.3-0.5)은 더 많은 블롭을 검출하지만 거짓 양성이 포함될 수 있습니다.")
    print("높은 임계값(0.6-0.8)은 더 선택적이어서 가장 강한 블롭 응답만 검출합니다.")
    print(f"이 해바라기 이미지의 경우, 임계값 {thresholds[blob_counts.index(max(blob_counts))]}에서 "
          f"가장 많은 블롭({max(blob_counts)}개)을 검출하지만, 시각적 검사 결과 "
          f"임계값 {thresholds[2]} 정도가 검출 수와 정확도 사이의 최적의 균형을 제공합니다.")

    # 최적의 매개변수 정의 및 비교
    param_sets = [
        {"sigmas": np.linspace(1, 20, 20), "filter_size": 31, "threshold": 0.6, "name": "초기 매개변수"},
        {"sigmas": np.linspace(2, 15, 25), "filter_size": 25, "threshold": 0.5, "name": "최적화된 매개변수"}
    ]

    # 시각화 설정
    plt.figure(figsize=(15, 8))

    # 각 매개변수 세트로 검출 실행 및 결과 표시
    for i, params in enumerate(param_sets):
        # 검출 실행
        response = trackScale(img_array, params["sigmas"], params["filter_size"], params["threshold"])

        # 시각화
        plt.subplot(1, 2, i+1)
        plt.imshow(response, cmap='viridis')
        plt.title(params["name"])
        plt.colorbar()
        plt.axis('off')

    plt.tight_layout()
    plt.show()

    # 매개변수 비교 출력
    print("매개변수 비교:")
    print("1. 초기 매개변수:")
    print(" - 시그마 범위: 1에서 20 (20개 값)")
    print(" - 필터 크기: 31 픽셀")
    print(" - 임계값: 0.6")
    print("\n2. 최적화된 매개변수:")
    print(" - 시그마 범위: 2에서 15 (25개 값)")
    print(" - 필터 크기: 25 픽셀")
    print(" - 임계값: 0.5")


def conclusion():
    """
    분석 결과와 최적의 매개변수에 대한 결론을 출력합니다.
    """
    print("===== 결론 =====")
    print("분석을 바탕으로, 해바라기 블롭 검출을 위한 최적의 매개변수는 다음과 같습니다:")

    print("1. 시그마 범위: 2에서 15까지 25개 값")
    print("   - 예상되는 해바라기 크기에 초점을 맞춘 좁은 범위")
    print("   - 이 범위 내에서 더 많은 값을 사용하여 스케일 정확도 향상")

    print("2. 필터 크기: 25 픽셀")
    print("   - 계산 효율성과 필터 커버리지 간의 좋은 균형")
    print("   - 30% 크기로 조정된 이미지에 적합")

    print("3. 임계값: 0.5")
    print("   - 검출률과 거짓 양성 사이의 균형 제공")
    print("   - 정확도를 유지하면서 상당한 수의 해바라기 검출")

    print("4. LoG 블롭 검출기는 밝은 꽃잎에 둘러싸인 어두운 원형 영역을 찾아 해바라기 중심을 성공적으로 식별")
    print("   - 스케일 공간 구현을 통해 이미지 내 다양한 크기와 깊이의 해바라기 검출 가능")


# 메인 실행 코드
if __name__ == "__main__":
    print("1. LoG 필터 함수 구현 및 시각화")
    display_LoG_filters()

    print("\n2. 이미지 필터링 함수 구현 완료")

    print("\n3. 해바라기 이미지 로드 및 전처리")
    img_array = load_and_preprocess_image()

    print("\n4. 최적의 매개변수로 블롭 검출")
    # 최적화된 매개변수
    sigmas = np.linspace(2, 15, 15)  # 예상되는 해바라기 크기에 초점을 맞춘 시그마 값 범위
    filter_size = 25  # 필터 크기
    threshold = 0.5  # 더 선택적인 검출을 위한 높은 임계값

    # 스케일 추적으로 블롭 검출
    filMaxScale = trackScale(img_array, sigmas, filter_size, threshold)

    print("\n5. 매개변수 최적화 분석")
    optimize_parameters(img_array)

    print("\n6. 결론")
    conclusion()